In [1]:
!kaggle datasets download -d tawsifurrahman/covid19-radiography-database

Dataset URL: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database
License(s): copyright-authors
100% 775M/778M [00:19<00:00, 43.4MB/s]
100% 778M/778M [00:19<00:00, 42.1MB/s]


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import zipfile



In [3]:
# ZIP dosyasının yolu
zip_file_path = '/content/covid19-radiography-database.zip'

# Çıkartılacak hedef klasör
extracted_folder_path = '/content/'

# ZIP dosyasını aç
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)



In [4]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split

data_dir = '/content/COVID-19_Radiography_Dataset'

train_dir = '/content/TrainTemp'
test_dir = '/content/Test'

classes = ['COVID', 'Normal', 'Viral Pneumonia']

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Her sınıf için eğitim ve test dizinlerini oluştur
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

# Her sınıf için görsel dosyalarını eğitim ve test dizinlerine kopyala
for cls in classes:
    cls_dir = os.path.join(data_dir, cls, 'images')
    images = os.listdir(cls_dir)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=99)

    for img in train_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(train_dir, cls, img)
        shutil.copy(src, dst)

    for img in test_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(test_dir, cls, img)
        shutil.copy(src, dst)

In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

train_dir = '/content/TrainTemp'

train_data_dir = '/content/Train'
validation_data_dir = '/content/Validation'
test_data_dir = '/content/Test'  # Test veri setinin bulunduğu dizin

os.makedirs(train_data_dir, exist_ok=True)
os.makedirs(validation_data_dir, exist_ok=True)

classes = ['COVID', 'Normal', 'Viral Pneumonia']

# Her sınıf için eğitim ve doğrulama dizinlerini oluştur
for cls in classes:
    os.makedirs(os.path.join(train_data_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(validation_data_dir, cls), exist_ok=True)

# Her sınıf için eğitim verisini eğitim ve doğrulama setlerine ayır
for cls in classes:
    cls_dir = os.path.join(train_dir, cls)
    train_images = os.listdir(cls_dir)
    train_images, validation_images = train_test_split(train_images, test_size=0.125, random_state=99)

    for img in train_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(train_data_dir, cls, img)
        shutil.copy(src, dst)

    for img in validation_images:
        src = os.path.join(cls_dir, img)
        dst = os.path.join(validation_data_dir, cls, img)
        shutil.copy(src, dst)


import shutil

shutil.rmtree("/content/TrainTemp")

In [6]:
# Veri setinin dizini
data_dir = "/content"

# Veri ön işleme ve artırma
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Veri yükleyicileri oluşturma
train_dataset = datasets.ImageFolder(root=data_dir + '/Train', transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = datasets.ImageFolder(root=data_dir + '/Test', transform=data_transforms)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)




In [7]:
class DeepFCN(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes):
        super(DeepFCN, self).__init__()
        self.fc_layers = nn.ModuleList()

        # İlk gizli katman
        self.fc_layers.append(nn.Linear(input_size, hidden_sizes[0]))
        self.fc_layers.append(nn.ReLU())

        # Ara gizli katmanlar
        for i in range(1, len(hidden_sizes)):
            self.fc_layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
            self.fc_layers.append(nn.ReLU())

        # Çıkış katmanı
        self.fc_layers.append(nn.Linear(hidden_sizes[-1], num_classes))

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Düzleştirme
        for layer in self.fc_layers:
            x = layer(x)
        return x



In [8]:
# Model parametreleri
input_size = 224 * 224 * 3  # Resim boyutu 224x224 ve 3 kanal (RGB)
hidden_sizes = [512, 256, 128]  # Gizli katman boyutları
num_classes = len(train_dataset.classes)

# Modeli oluştur
model = DeepFCN(input_size, hidden_sizes, num_classes)

# Eğitim için CUDA kullanılabilirse GPU'ya taşıma
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()



In [9]:
from tqdm import tqdm

# Epoch sayısını 1 olarak ayarlama
epochs = 3
for epoch in range(epochs):
    running_loss = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", unit="batch", colour='green', leave=True)
    for batch_idx, (inputs, labels) in progress_bar:
        inputs, labels = inputs.to(device), labels.to(device)

        # Gradientleri sıfırlama
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass ve optimizasyon
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        progress_bar.set_postfix({'Loss': loss.item()})

    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}")



Epoch 1/3: 100%|██████████| 332/332 [00:48<00:00,  6.85batch/s, Loss=0.428]


Epoch 1/3, Loss: 0.6160


Epoch 2/3: 100%|██████████| 332/332 [00:49<00:00,  6.77batch/s, Loss=0.466]


Epoch 2/3, Loss: 0.4875


Epoch 3/3: 100%|██████████| 332/332 [00:47<00:00,  6.95batch/s, Loss=0.497]

Epoch 3/3, Loss: 0.4390


In [10]:
correct = 0
total = 0
TP = 0
FP = 0
FN = 0
TN = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Kesinlik ve Duyarlılık için TP, FP, FN ve TN değerlerini hesapla
        TP += ((predicted == labels) & (labels == 1)).sum().item()
        FP += ((predicted != labels) & (labels == 0)).sum().item()
        FN += ((predicted != labels) & (labels == 1)).sum().item()
        TN += ((predicted == labels) & (labels == 0)).sum().item()

# Doğruluk (Accuracy) hesapla
accuracy = correct / total

# Kesinlik (Precision) hesapla
precision = TP / (TP + FP) if (TP + FP) != 0 else 0

# Duyarlılık (Recall) hesapla
recall = TP / (TP + FN) if (TP + FN) != 0 else 0

# F1-Skoru hesapla
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")

print(f"TP: {TP}")
print(f"FP: {FP}")
print(f"FN: {FN}")
print(f"TN: {TN}")

Test Accuracy: 0.7925
Precision: 0.8159
Recall: 0.9691
F1-Score: 0.8859
TP: 1976
FP: 446
FN: 63
TN: 278
